In [2]:
def jpeg_encode(block_size, quantization_matrix,xray):
    h,w = xray.shape
    height = np.int32(h)
    width = np.int32(w)
    h,w = np.float32(h),np.float32(w)

    nbh = np.int32(math.ceil(h/block_size))
    nbw = np.int32(math.ceil(w/block_size))

    #Height of padded image
    H = block_size * nbh
    W = block_size * nbw

    padded_img = np.zeros((H,W))
    padded_img[0:height,0:width] = xray[0:height,0:width]

    padded_img_minus128 = padded_img - 128
    
    final = np.zeros(xray.shape)
    for i in range(nbh):
        #compute start and end row index of block
        row_ind_1 = i*block_size
        row_ind_2 = row_ind_1 + block_size
    
    
        for j in range(nbw):
            #compute start and end column index of block
            col_ind_1 = j*block_size
            col_ind_2 = col_ind_1 + block_size
        
            block = padded_img_minus128[row_ind_1 : row_ind_2, col_ind_1:col_ind_2]
        
            #applying 2D DCT to block
            DCT = cv2.dct(block)
        
            quantized = np.round(np.divide(DCT,quantization_matrix))
        
            reordered = zigzag(quantized)
        
            #reshaped = np.reshape(reordered,(block_size,block_size))
        
            final[row_ind_1:row_ind_2,col_ind_1:col_ind_2] = reordered
            
    arranged = final.flatten()
            
    bitstream = run_length_encode(arranged)
            
    bitstream2 = str(padded_img.shape[0]) + " " + str(padded_img.shape[1]) + " " + bitstream + ";"
            
    file1 = open('test.txt','w')
    file1.write(bitstream2)
    file1.close()
    

In [7]:
def jpeg_decode(bitstream,xray,quantization_matrix,block_size):
    decoded = rl_decode(bitstream,xray.shape)
    uncompressed = np.zeros(decoded.shape)
    for i in range(nbh):
        #compute start and end row index of block
        row_ind_1 = i*block_size
        row_ind_2 = row_ind_1 + block_size
    
    
    
        for j in range(nbw):
            #compute start and end column index of block
            col_ind_1 = j*block_size
            col_ind_2 = col_ind_1 + block_size
        
            block = decoded[row_ind_1 : row_ind_2, col_ind_1:col_ind_2]
        
            #applying 2D IDCT to block
        
            reordered = reverse_zigzag_for8x8(block)
        
            dequantized_block = np.round(np.multiply(block,quantization_matrix)).astype(np.float64)
            IDCT = cv2.idct(dequantized_block)
        
            #reshaped = np.reshape(reordered,(block_size,block_size))
        
            uncompressed[row_ind_1:row_ind_2,col_ind_1:col_ind_2] = IDCT
    
    uncompressed2 =np.uint8(uncompressed) + 128

In [4]:
def compare(img1,img1_jpeg):
    plt.subplot(1,2,1)
    plt.imshow(img1,cmap='gray')
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(img1_jpeg,cmap='gray')
    plt.axis('off')

In [5]:
def rmse(img1,img2):
    mse = np.square(np.subtract(img1,img2)).mean()
    rmse = math.sqrt(mse)
    return rmse

def snr(filtered_img,original_img):
    num = np.square(filtered_img).sum()
    diff_sq = np.square(filtered_img - original_img)
    denom = np.sum(diff_sq)
    snr = num/denom
    return snr

In [6]:
def print_metrics(jpeg,xray):
    rmse_jpeg = rmse(jpeg,xray)
    snr_jpeg = snr(jpeg,xray)
    print(f"For JPEG: RMSE = {rmse_jpeg} | SNR = {snr_jpeg}")

In [8]:
def zigzag(matrix):
    # Get the number of rows and columns in the matrix
    rows = len(matrix)
    cols = len(matrix[0])

    # Create an empty list to store the zigzag ordering
    zigzag = []
    temp = np.zeros((1,rows*cols))

    # Initialize the starting row and column
    row, col = 0, 0

    # Iterate over each diagonal of the matrix
    for i in range(rows + cols - 1):
        # If the diagonal is even, traverse it from top-right to bottom-left
        if i % 2 == 0:
            while row >= 0 and col < cols:
                zigzag.append(matrix[row][col])
                row -= 1
                col += 1
            if row < 0 and col <= cols - 1:
                row = 0
            if col == cols:
                row += 2
                col -= 1
        # If the diagonal is odd, traverse it from bottom-left to top-right
        else:
            while col >= 0 and row < rows:
                zigzag.append(matrix[row][col])
                row += 1
                col -= 1
            if col < 0 and row <= rows - 1:
                col = 0
            if row == rows:
                col += 2
                row -= 1
    zigzag = np.array(zigzag)
    zigzag = np.reshape(zigzag,(-1,matrix.shape[0]))
    
    return zigzag

In [9]:
def reverse_zigzag_for8x8(matrix):
    for i in range(135):
        matrix= zigzag(matrix)
    return matrix

In [10]:
def reverse_zigzag_for4x4(matrix):
    for i in range(5):
        matrix = zigzag(matrix)
    return matrix

In [11]:
def run_length_encode(input_string):
    # Initialize the encoded string and the counter
    encoded_string = ''
    count = 1

    # Iterate over each character in the input string
    for i in range(1, len(input_string)):
        if input_string[i] == input_string[i-1]:
            # If the current character is the same as the previous one, increment the counter
            count += 1
        else:
            # If the current character is different from the previous one, add the count and the character to the encoded string
            count_str = str(count)
            encoded_string += count_str + "," +str(input_string[i-1]) + "."
            count = 1

    # Add the final count and character to the encoded string
    encoded_string += str(count) +"," + str(input_string[-1]) 

    return encoded_string   

In [12]:
def rl_decode(g,shape):

    l = g.split(".")
    final = []
    for i in l:
        temp = i.split(',')
        final.append(np.int32(temp))
    
    #final
    final_arr = []
    for i in final:
        for j in range(i[0]):
            final_arr.append(i[1])
        
    k = np.array(final_arr)

    return k.reshape(shape)